# Day 2: Matrix Transpose (Coalesced vs Non-Coalesced)

In this notebook, we compare the performance of a matrix transpose operation implemented using two different CUDA kernels: one with non-coalesced access and one optimized for coalesced access using shared memory. We'll compile and run the CUDA code on Google Colab's T4 GPU.

In [13]:
%%writefile matrix_transpose.cu

#include <iostream>
#include <cstdlib>
#include <cuda_runtime.h>
#include <chrono>

#define TILE_DIM 32
#define BLOCK_ROWS 8

// Non-coalesced Matrix Transpose Kernel
__global__ void transposeNonCoalesced(const float *in, float *out, int width, int height) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < height && col < width) {
        // Writing to the output in a way that results in non-coalesced accesses
        out[col * height + row] = in[row * width + col];
    }
}

// Coalesced Matrix Transpose Kernel using shared memory
__global__ void transposeCoalesced(const float *in, float *out, int width, int height) {
    // Shared memory tile with an extra column to avoid bank conflicts
    __shared__ float tile[TILE_DIM][TILE_DIM+1];

    int x = blockIdx.x * TILE_DIM + threadIdx.x;
    int y = blockIdx.y * TILE_DIM + threadIdx.y;

    if (x < width && y < height) {
        tile[threadIdx.y][threadIdx.x] = in[y * width + x];
    }

    __syncthreads();

    // Compute transposed indices
    x = blockIdx.y * TILE_DIM + threadIdx.x; // swap blockIdx.x and blockIdx.y
    y = blockIdx.x * TILE_DIM + threadIdx.y;

    if (x < height && y < width) {
        out[y * height + x] = tile[threadIdx.x][threadIdx.y];
    }
}

int main(int argc, char** argv) {
    // Get matrix dimensions from command-line arguments
    int width = 1024;  // Default width
    int height = 1024; // Default height

    if (argc > 2) {
        width = atoi(argv[1]);
        height = atoi(argv[2]);
    }
    int size = width * height;
    size_t bytes = size * sizeof(float);

    // Allocate host memory
    float *h_in = (float*)malloc(bytes);
    float *h_out = (float*)malloc(bytes);

    // Initialize input matrix
    for(int i = 0; i < size; i++) {
        h_in[i] = static_cast<float>(i);
    }

    // Allocate device memory
    float *d_in, *d_out;
    cudaMalloc(&d_in, bytes);
    cudaMalloc(&d_out, bytes);

    cudaMemcpy(d_in, h_in, bytes, cudaMemcpyHostToDevice);

    // Configure grid and block dimensions
    dim3 block(TILE_DIM, BLOCK_ROWS);
    dim3 grid((width + TILE_DIM - 1) / TILE_DIM, (height + TILE_DIM - 1) / TILE_DIM);

    // Run Non-Coalesced Transpose Kernel
    auto start = std::chrono::high_resolution_clock::now();
    transposeNonCoalesced<<<grid, block>>>(d_in, d_out, width, height);
    cudaDeviceSynchronize();
    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> duration_non_coalesced = end - start;

    // Run Coalesced Transpose Kernel
    start = std::chrono::high_resolution_clock::now();
    transposeCoalesced<<<grid, block>>>(d_in, d_out, width, height);
    cudaDeviceSynchronize();
    end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> duration_coalesced = end - start;

    // Print the results
    std::cout << "Non-Coalesced Time: " << duration_non_coalesced.count() << " s" << std::endl;
    std::cout << "Coalesced Time: " << duration_coalesced.count() << " s" << std::endl;

    // Free allocated memory
    cudaFree(d_in);
    cudaFree(d_out);
    free(h_in);
    free(h_out);

    return 0;
}


Overwriting matrix_transpose.cu


In [ ]:
!nvcc matrix_transpose.cu -o matrix_transpose

In [ ]:
import matplotlib.pyplot as plt
import subprocess
import numpy as np
import time

# Function to run the CUDA code and get execution times
def run_transpose_cuda(width, height):
    command = ["./matrix_transpose", str(width), str(height)]
    output = subprocess.check_output(command).decode("utf-8")
    lines = output.split("\n")
    non_coalesced_time = float(lines[0].split(":")[1].strip().split(" ")[0])
    coalesced_time = float(lines[1].split(":")[1].strip().split(" ")[0])
    return non_coalesced_time, coalesced_time

# Function to perform CPU-based matrix transpose
def transpose_cpu(matrix):
    start_time = time.time()
    transposed_matrix = matrix.T  # Using NumPy's transpose function
    end_time = time.time()
    return end_time - start_time

# Matrix dimensions to test
dimensions = [(256, 256), (512, 512), (1024, 1024), (2048, 2048)]

# Store execution times
non_coalesced_times = []
coalesced_times = []
cpu_times = []

# Run the transpose for different matrix sizes
for width, height in dimensions:
    # CUDA transpose
    non_coalesced_time, coalesced_time = run_transpose_cuda(width, height)
    non_coalesced_times.append(non_coalesced_time)
    coalesced_times.append(coalesced_time)

    # CPU transpose
    matrix = np.random.rand(width, height)  # Create a random matrix
    cpu_time = transpose_cpu(matrix)
    cpu_times.append(cpu_time)

# Extract widths for plotting
widths = [dim[0] for dim in dimensions]

# Plot the results
plt.plot(widths, non_coalesced_times, label="Non-Coalesced (CUDA)")
plt.plot(widths, coalesced_times, label="Coalesced (CUDA)")
plt.plot(widths, cpu_times, label="CPU")
plt.xlabel("Matrix Dimension")
plt.ylabel("Log of Execution Time (s)")
plt.title("Coalesced vs Non-Coalesced vs CPU Matrix Transpose")
plt.legend()

# Adjust y-axis scale
plt.yscale("log")  # Change to logarithmic scale

plt.show()

In [8]:
!./matrix_transpose 128 256  # Example: width = 128, height = 256

Non-Coalesced Time: 0.00778203 s
Coalesced Time: 2.153e-06 s
